In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
pip install mxnet

In [ ]:
import mxnet as mx
import numpy as np
import pandas as pd
import seaborn as sns
import nltk
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer, LabelEncoder
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:

df=pd.read_csv('../input/bbc-text.csv')

In [ ]:
df.head(10)

In [ ]:
df.shape

In [ ]:
df['category'].unique()

In [ ]:
print(df['category'].value_counts().sort_values)

In [ ]:
sns.countplot(df['category'])
plt.show()

In [ ]:
import nltk
nltk.download('punkt')

In [ ]:
from nltk.tokenize import word_tokenize
df['text'] = df.apply(lambda row: nltk.word_tokenize(row['text']), axis=1)

In [ ]:
encoder = LabelEncoder()
encoder.fit(df['category'])
df['category']=encoder.transform(df['category'])

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(df['text'], df['category'],test_size=0.3,random_state=0)

In [ ]:
x_train

In [ ]:
y_train.shape

In [ ]:
x_test.shape

In [ ]:
type(x_test)

In [ ]:
import multiprocessing

from mxnet import nd, autograd, gluon
from mxnet.gluon.data import ArrayDataset
from mxnet.gluon.data import DataLoader
import numpy as np

In [ ]:
train_dataloader = DataLoader(
    x_train,
    batch_size=128,
    num_workers=max(multiprocessing.cpu_count() - 3, 1),
    last_batch='rollover'
)

In [ ]:
test_dataloader = DataLoader(
    x_test,
    shuffle=False,
    batch_size=128,
    num_workers=max(multiprocessing.cpu_count() - 3, 1),
    last_batch='rollover'
)

In [ ]:
net = gluon.nn.HybridSequential()                                                               
with net.name_scope():
    net.add(gluon.nn.Conv1D(channels=256 , kernel_size=7, activation='relu'))                   #Convolutional Layers
    net.add(gluon.nn.MaxPool1D(pool_size=3, strides=3))
    net.add(gluon.nn.Flatten())
    net.add(gluon.nn.Dropout(0.5))
    net.add(gluon.nn.Dense(5))

In [ ]:
ctx=mx.cpu()
net.add(gluon.nn.Dropout(0.5))

In [ ]:
print(net)

In [ ]:
net.hybridize(static_alloc=True, static_shape=True)

In [ ]:
softmax_cross_entropy = gluon.loss.SoftmaxCrossEntropyLoss()

In [ ]:
def evaluate_accuracy(data_iterator, net):
    acc = mx.metric.Accuracy()
    for i, (df, label) in enumerate(data_iterator):
        df = d.as_in_context(ctx)
        label = label.as_in_context(ctx)
        output = net(data)
        prediction = nd.argmax(output, axis=1)
        acc.update(preds=prediction, labels=label)
    return acc.get()[1]

In [ ]:
import random
for i in range(50):
    index = random.randint(1, len(df))
    text = df['text'][index]
    label = df['category']
    print(text)
    print('\nCategory: {}\n'.format(label))
    encoded = nd.array([encode(text)], ctx=ctx)
    output = net(encoded)
    predicted = categories[np.argmax(output[0].asnumpy())]
    if predicted == label:
          print('Correct')
    else:
          print('Incorrectly predicted {}'.format(predicted))